In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [43]:
pd.set_option('display.float_format', '{:.6f}'.format)

In [44]:
df = pd.read_csv(r"C:\Year Three\Semester One\Artificial Neural Network\Dataset\HI-Large_Trans_Updated.csv")
df

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/11/01 14:36,193455,822C47980,94047,8230E3120,4371.840000,UK Pound,4371.840000,UK Pound,ACH,1
1,2022/11/09 14:12,5475,806D8B2E0,112934,80521E600,10045.170000,US Dollar,10045.170000,US Dollar,ACH,1
2,2022/10/10 01:33,24555,802E684A0,236302,80E44DA60,59792.450000,US Dollar,59792.450000,US Dollar,ACH,0
3,2022/08/24 14:41,262462,817FD3CA0,159640,817FD6200,17305.520000,Rupee,17305.520000,Rupee,ACH,1
4,2022/09/19 23:48,26,81A2CA100,78843,81D6BED70,14.080000,Ruble,14.080000,Ruble,ACH,0
...,...,...,...,...,...,...,...,...,...,...,...
451087,2022/09/26 16:31,215233,815AE8170,4339,8020E3650,15896.590000,Euro,15896.590000,Euro,ACH,1
451088,2022/09/22 22:25,22054,801ECD000,22054,801ECD000,359.000000,Canadian Dollar,232.220000,Euro,ACH,0
451089,2022/10/28 09:17,216360,81108AD30,44211,8119E1FB0,34330.060000,Yuan,34330.060000,Yuan,Cheque,0
451090,2022/09/28 15:18,70,1004288A0,9,82E0F3270,2104.040000,Australian Dollar,2104.040000,Australian Dollar,Cheque,1


In [45]:
df['Is Laundering'].value_counts()

Is Laundering
1    225546
0    225546
Name: count, dtype: int64

In [46]:
balanced_df = df.copy()
balanced_df

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/11/01 14:36,193455,822C47980,94047,8230E3120,4371.840000,UK Pound,4371.840000,UK Pound,ACH,1
1,2022/11/09 14:12,5475,806D8B2E0,112934,80521E600,10045.170000,US Dollar,10045.170000,US Dollar,ACH,1
2,2022/10/10 01:33,24555,802E684A0,236302,80E44DA60,59792.450000,US Dollar,59792.450000,US Dollar,ACH,0
3,2022/08/24 14:41,262462,817FD3CA0,159640,817FD6200,17305.520000,Rupee,17305.520000,Rupee,ACH,1
4,2022/09/19 23:48,26,81A2CA100,78843,81D6BED70,14.080000,Ruble,14.080000,Ruble,ACH,0
...,...,...,...,...,...,...,...,...,...,...,...
451087,2022/09/26 16:31,215233,815AE8170,4339,8020E3650,15896.590000,Euro,15896.590000,Euro,ACH,1
451088,2022/09/22 22:25,22054,801ECD000,22054,801ECD000,359.000000,Canadian Dollar,232.220000,Euro,ACH,0
451089,2022/10/28 09:17,216360,81108AD30,44211,8119E1FB0,34330.060000,Yuan,34330.060000,Yuan,Cheque,0
451090,2022/09/28 15:18,70,1004288A0,9,82E0F3270,2104.040000,Australian Dollar,2104.040000,Australian Dollar,Cheque,1


In [47]:
min_count = df['Is Laundering'].value_counts().min()
balanced_df = df.groupby('Is Laundering').apply(
    lambda x: x.sample(min_count, random_state=42)
).reset_index(drop=True)

balanced_df['Is Laundering'].value_counts()

In [48]:
balanced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451092 entries, 0 to 451091
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Timestamp           451092 non-null  object 
 1   From Bank           451092 non-null  int64  
 2   Account             451092 non-null  object 
 3   To Bank             451092 non-null  int64  
 4   Account.1           451092 non-null  object 
 5   Amount Received     451092 non-null  float64
 6   Receiving Currency  451092 non-null  object 
 7   Amount Paid         451092 non-null  float64
 8   Payment Currency    451092 non-null  object 
 9   Payment Format      451092 non-null  object 
 10  Is Laundering       451092 non-null  int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 37.9+ MB


In [49]:
# balanced_df.to_csv("HI-Large_Trans_Updated.csv", index=True)

In [50]:
#device = torch.device('cuda')

In [51]:
#torch.cuda.is_available()

In [52]:
balanced_df['Timestamp'] = pd.to_datetime(balanced_df['Timestamp'])
balanced_df['Timestamp'] = balanced_df['Timestamp'].astype('int64')

time_min = balanced_df['Timestamp'].min()
time_max = balanced_df['Timestamp'].max()
balanced_df['Timestamp'] = (balanced_df['Timestamp'] - time_min)/(time_max - time_min)

# amount_min = balanced_df['Amount Paid'].min()
# amount_max = balanced_df['Amount Paid'].max()
# balanced_df['Amount Paid'] = (balanced_df['Amount Paid'] - amount_max)/(amount_max - amount_min)

balanced_df

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,0.562283,193455,822C47980,94047,8230E3120,4371.840000,UK Pound,4371.840000,UK Pound,ACH,1
1,0.610754,5475,806D8B2E0,112934,80521E600,10045.170000,US Dollar,10045.170000,US Dollar,ACH,1
2,0.425406,24555,802E684A0,236302,80E44DA60,59792.450000,US Dollar,59792.450000,US Dollar,ACH,0
3,0.143362,262462,817FD3CA0,159640,817FD6200,17305.520000,Rupee,17305.520000,Rupee,ACH,1
4,0.303530,26,81A2CA100,78843,81D6BED70,14.080000,Ruble,14.080000,Ruble,ACH,0
...,...,...,...,...,...,...,...,...,...,...,...
451087,0.344189,215233,815AE8170,4339,8020E3650,15896.590000,Euro,15896.590000,Euro,ACH,1
451088,0.321395,22054,801ECD000,22054,801ECD000,359.000000,Canadian Dollar,232.220000,Euro,ACH,0
451089,0.536651,216360,81108AD30,44211,8119E1FB0,34330.060000,Yuan,34330.060000,Yuan,Cheque,0
451090,0.356025,70,1004288A0,9,82E0F3270,2104.040000,Australian Dollar,2104.040000,Australian Dollar,Cheque,1


In [53]:
label_account = LabelEncoder()
label_currency = LabelEncoder()
label_format = LabelEncoder()

In [54]:
all_account = pd.concat([balanced_df['Account'],balanced_df['Account.1']]).unique()
label_account.fit(all_account)

LabelEncoder()

In [55]:
sender = label_account.transform(balanced_df['Account'])
receiver = label_account.transform(balanced_df['Account.1'])
edge_index = torch.tensor([sender,receiver], dtype=torch.long)

In [56]:
balanced_df['Payment Currency'] = label_currency.fit_transform(balanced_df['Payment Currency'])
balanced_df['Payment Format'] = label_format.fit_transform(balanced_df['Payment Format'])

In [57]:
feature_columns = ['Amount Paid', 'Payment Currency', 'Payment Format', 'Timestamp']
edge_attr = torch.tensor(balanced_df[feature_columns].values,dtype=torch.float)

In [58]:
y = torch.tensor(balanced_df['Is Laundering'].values, dtype=torch.float)
number_of_nodes = len(all_account)
x = torch.ones((number_of_nodes,1), dtype=torch.float)

In [59]:
# --- STEP 1: CONVERT LABELS FROM TRANSACTIONS TO NODES ---
# Your current 'y' describes edges. We need 'node_y' to describe accounts.

# Create a blank label list for all accounts (Default 0 = Legal)
num_nodes = x.shape[0]
node_y = torch.zeros(num_nodes, dtype=torch.long) # Use .long() for CrossEntropyLoss

# Find which transactions were Money Laundering (1)
laundering_indices = (y == 1).nonzero(as_tuple=True)[0]

# Find the Sender and Receiver IDs involved in those bad transactions
bad_senders = edge_index[0, laundering_indices]
bad_receivers = edge_index[1, laundering_indices]

# Mark those accounts as "1" (Suspicious)
node_y[bad_senders] = 1
node_y[bad_receivers] = 1

# --- STEP 2: CREATE THE MASKS (SPLIT THE DATA) ---
# We will split the ACCOUNTS (Nodes), not the transactions.
# 80% Train, 10% Validation, 10% Test

# Create a random order of accounts
perm = torch.randperm(num_nodes)

# Calculate split sizes
n_train = int(num_nodes * 0.8)
n_val = int(num_nodes * 0.1)
# The rest goes to test

# Initialize masks as all False
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

# Assign True based on the random permutation
train_mask[perm[:n_train]] = True
val_mask[perm[n_train:n_train+n_val]] = True
test_mask[perm[n_train+n_val:]] = True

# --- STEP 3: UPDATE THE DATA OBJECT ---
# We replace the old 'y' with 'node_y' so sizes match [num_nodes]
data = Data(
    x=x, 
    edge_index=edge_index, 
    edge_attr=edge_attr, 
    y=node_y, 
    train_mask=train_mask, 
    val_mask=val_mask, 
    test_mask=test_mask
)

In [60]:
torch.save(data, 'aml_graph_large.pt')

In [61]:
data

Data(x=[486746, 1], edge_index=[2, 451092], edge_attr=[451092, 4], y=[486746], train_mask=[486746], val_mask=[486746], test_mask=[486746])